# CMIP5 Global mean temperatures

Create a netCDF with historical, and projected climatlological precipitation, as well as the relative change between them, from all CMIP5 models for RCP8.5 (Taylor et al., 2012). We make use of Jan Sedláček's post-processing of CMIP5 data (https://data.iac.ethz.ch/atmos/). The data is stored at IAC at: /net/atmos/data/cmip5-ng/.

The table with all used models is at the end of the [notebook](#Models).

In [ ]:
from glob import glob
from os import path

import xarray as xr

import pandas as pd
import numpy as np

from scipy import stats

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
%matplotlib inline

In [ ]:
def cmip5_filename(var, time, model, scen, ens, res='g025'):
    """
    list cmip5 filenames according to criteria

    Parameters
    ----------
    var : string
        Variable name.
    time : string
        Time resolution, e.g. 'ann', 'seas'.
    model : string
        Models to look for, e.g. '*', 'NorESM1'
    scen : string
        Scenario, e.g. 'rcp85', ...
    ens : string
        Which ensemble members, e.g. '*', 'r1i1p?', 'r1i1p1'
    res : string
        Resolution, 'native' or 'g025'. Optional, default: 'g025'.

    ..note::

    All arguments can take wildcards.

    """

    folder_root = '/net/atmos/data/cmip5-ng/'
    folder = path.join(folder_root, var)

    fN = '_'.join([var, time, model, scen, ens, res])
    fN = path.join(folder, fN + '.nc')
    
    fN = sorted(glob(fN))

    if not fN:
        raise RuntimeError('No simulations found')

    return fN



In [ ]:
# obtain a list of cmip5 output, annual mean, surface air temperature
# use only the first ensemble member

fNs = cmip5_filename('pr', 'ann', '*', 'rcp85', 'r1i1p1', res='g025')

print(len(fNs))

for fN in fNs[:4]:
    print(fN)
    
print('...')


In [ ]:
all_models = []
for fN in fNs:
    basename = path.basename(fN)
    all_models.append(basename.split('_')[2])

In [ ]:
# read all models

hist = list()
proj = list()


for fN in fNs:

    # open datset
    ds = xr.open_dataset(fN)
    ds.load()

    # calculate climatology
    h = ds.sel(year=slice('1986', '2005')).mean('year')
    p = ds.sel(year=slice('2081', '2100')).mean('year')
    
    # append to list
    hist.append(h)
    proj.append(p)

    
# concatenate    
hist = xr.concat(hist, dim='ens')
proj = xr.concat(proj, dim='ens')



In [ ]:
ds = (100 * (proj.mean('ens') - hist.mean('ens')) / hist.mean('ens'))

ds = ds.rename(dict(pr='pr_rel'))

In [ ]:
ds

In [ ]:
ds = ds.assign(proj=proj.pr.mean('ens') * 24 * 3600 * 365)
ds = ds.assign(hist=hist.pr.mean('ens') * 24 * 3600 * 365)

In [ ]:
ds.attrs['data'] = 'CMIP5 precipitation (pr)'
ds.attrs['time_periods'] = 'hist: 1986-2005; proj: 2081-2100'
ds.attrs['pr_delta'] = '(proj - hist) / hist * 100'

ds.attrs['source'] = 'https://data.iac.ethz.ch/atmos/'
ds.attrs['reference'] = 'Taylor et al., 2012'
ds.attrs['scenario'] = 'rcp8.5'

ds.attrs['interpolation_grid'] = '2.5 x 2.5 degrees bilinear interpolation'

ds.pr_rel.attrs['units'] = '%'

ds.proj.attrs['units'] = 'mm'
ds.hist.attrs['units'] = 'mm'

In [ ]:
(hist.mean('ens') * 24 * 3600 * 365).pr.plot(infer_intervals=False)

In [ ]:
# plot relative changes

ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()

ds.pr_rel.plot(transform=ccrs.PlateCarree(), vmax=50, center=0, extend='both', cmap='BrBG', infer_intervals=False)

ax.set_global()

ds.lon

In [ ]:
# calculate the percentage of models that show a increase / decrease

# the sign indicates if pr in- or decreases
s = np.sign(proj.pr - hist.pr)

# count the number of models that agree
n_incr = (s == -1).sum('ens')
n_decr = (s == 1).sum('ens')

# we need the maximum percentage
agree_sign = np.fmax((n_incr / 40), (n_decr / 40)) * 100

ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
agree_sign.plot(ax=ax)


agree_sign.name = 'agree_sign'
agree_sign.attrs['units'] = '%'

ds = ds.assign(agree_sign=agree_sign)
ds

In [ ]:
# calculate significance using a t-test

axis = proj.pr.get_axis_num('ens')

_, pval = stats.ttest_ind(proj.pr, hist.pr, axis=axis)


ds = ds.assign(pval=(('lat', 'lon'), pval))


ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()

ds.pval.plot(vmin=0, vmax=1)


ds

In [ ]:
ds.to_netcdf('cmip5_delta_pr_rcp85_map.nc', format='NETCDF4_CLASSIC')

## Models

Print all models:

In [ ]:
for model in all_models:
    print(model)